In [7]:
from initial_functions import fn_write_to_db

In [8]:
# pip install websocket-client

In [11]:
API_KEY = 'd60bqkhr01qgk0vil07gd60bqkhr01qgk0vil080'

In [ ]:
import websocket
import json
import pandas as pd
from datetime import datetime, timedelta
from time import sleep

# -----------------------------
# 1️⃣ Kullanıcı Ayarları
# -----------------------------
SYMBOLS = ["AAPL", "TSLA", "MSFT", "GOOGL"]
PERIOD_MINUTES = 1  # 1, 15, 30, 60 dakika
TABLE_NAME = "streamdata"  # DB tablosu

# Kendi DB yazma fonksiyonun
# fn_write_to_db(df=df, table_name='streamdata', if_exist='append')

# -----------------------------
# 2️⃣ Global değişkenler
# -----------------------------
latest_price = {}     # her sembol için son fiyat ve timestamp
trade_count = {}      # her sembol için periyot boyunca trade sayısı
volume_latest = {}    # her sembol için son gelen volume (varsa)

last_write_time = datetime.now()

# -----------------------------
# 3️⃣ WebSocket Callback Fonksiyonları
# -----------------------------
def on_open(ws):
    print("✅ WebSocket bağlandı!")
    for s in SYMBOLS:
        ws.send(json.dumps({
            "type": "subscribe",
            "symbol": s
        }))

def on_message(ws, message):
    global latest_price, trade_count, volume_latest, last_write_time

    try:
        data = json.loads(message)
        if "data" in data:
            for trade in data["data"]:
                symbol = trade["s"]
                price = trade["p"]
                ts = datetime.fromtimestamp(trade["t"] / 1000)  # milisaniye -> saniye
                vol = trade.get("v", None)  # volume varsa al, yoksa None

                # son fiyat kaydet
                latest_price[symbol] = {"symbol": symbol, "price": price, "timestamp": ts}

                # son volume kaydet (varsa)
                if vol is not None:
                    volume_latest[symbol] = vol

                # trade count artır
                trade_count[symbol] = trade_count.get(symbol, 0) + 1

        # periyot kontrolü
        now = datetime.now()
        if (now - last_write_time) >= timedelta(minutes=PERIOD_MINUTES):
            if latest_price:
                df_to_write = pd.DataFrame([
                    {
                        "symbol": s,
                        "price": latest_price[s]["price"],
                        "timestamp": latest_price[s]["timestamp"],
                        "volume": volume_latest.get(s, None),
                        "trade_count": trade_count.get(s, 0)
                    }
                    for s in latest_price
                ])

                # DB'ye yaz
                fn_write_to_db(df_to_write, TABLE_NAME, 'append')

                # periyot sonunda trade_count reset
                trade_count = {}

            last_write_time = now

    except Exception as e:
        print("❌ Hata:", e)

def on_error(ws, error):
    print("HATA:", error)

def on_close(ws):
    print("WebSocket kapandı!")

# -----------------------------
# 4️⃣ WebSocket Başlat
# -----------------------------
socket_url = f"wss://ws.finnhub.io?token={API_KEY}"

ws = websocket.WebSocketApp(
    socket_url,
    on_open=on_open,
    on_message=on_message,
    on_error=on_error,
    on_close=on_close
)

# -----------------------------
# 5️⃣ Sürekli çalıştır
# -----------------------------
while True:
    try:
        ws.run_forever()
    except KeyboardInterrupt:
        print("Program durduruldu!")
        break
    except Exception as e:
        print("WebSocket yeniden bağlanıyor...", e)
        sleep(5)


✅ WebSocket bağlandı!
*** SAVED TO DB: streamdata | append
*** SAVED TO DB: streamdata | append
*** SAVED TO DB: streamdata | append
*** SAVED TO DB: streamdata | append
*** SAVED TO DB: streamdata | append
*** SAVED TO DB: streamdata | append
